# Data Preproccessing and Writing to CSV for Big Dataset (70K+ Tweets) from Twitter API

In [1]:
# Import dependencies
import tweepy
import pandas as pd
import numpy as np
import json
import time
from datetime import datetime
import addfips

In [2]:
# Code reproduced from Twitter developer API documentation. Additional parsing features and customizations made where necessary for desired output.
def determine_tweet_type(tweet):
    # Check for reply indicator first
    if tweet["in_reply_to_status_id"] is not None:
        tweet_type = "Reply Tweet"
    # Check boolean quote status field but make sure it's not a Retweet (of a Quote Tweet) 
    elif tweet["is_quote_status"] is True and not tweet["text"].startswith("RT"):
        tweet_type = "Quote Tweet"
    # Check both indicators of a Retweet
    elif tweet["text"].startswith("RT") and tweet.get("retweeted_status") is not None:
        tweet_type = "Retweet"
    else:
        tweet_type = "Original Tweet"
    return tweet_type

parsedTweets = []

def parse_tweets(status):
    for tweet in status:
        
        hashtags = []
        mentions = []
        

        if 'extended_tweet' in tweet:
            full_text = tweet['extended_tweet']['full_text']

            # Collect hashtags from tweet into list
            hashtags_entity = tweet["extended_tweet"]["entities"]["hashtags"]
            if len(hashtags_entity) == 0:
                hashtags = None
            else:
                for hashtag in hashtags_entity:
                    hashtags.append("#" + hashtag['text'])

            # Collect mentions from tweet into list
            mentions_entity = tweet["extended_tweet"]["entities"]["user_mentions"]
            if len(mentions_entity) == 0:
                mentions = None
            else:
                for mention in mentions_entity:
                    mentions.append("@" + mention["screen_name"])

        else:
            full_text = tweet['text']

            # Collect hashtags from tweet into list
            hashtags_entity = tweet["entities"]["hashtags"]
            if len(hashtags_entity) == 0:
                hashtags = None
            else:
                for hashtag in hashtags_entity:
                    hashtags.append("#" + hashtag['text'])

            # Collect mentions from tweet into list
            mentions_entity = tweet["entities"]["user_mentions"]
            if len(mentions_entity) == 0:
                mentions = None
            else:
                for mention in mentions_entity:
                    mentions.append("@" + mention["screen_name"])
        
                
        # Get coordinates latitude and longitude into separate variables by subscripting coordinates if coordinates object not Nonetype
        coordinates = tweet["coordinates"]
        if coordinates is not None:
            coord_lat = tweet["coordinates"]["coordinates"][0]
            coord_lng = tweet["coordinates"]["coordinates"][1]
        else:
            coord_lat = None
            coord_lng = None
            
        
        # Get place attributes into separate variables by subscripting place if place object not Nonetype 
        place = tweet["place"]
        if place is not None:
            place_type = tweet["place"]["place_type"]
            place_name = tweet["place"]["name"]
            place_full_name = tweet["place"]["full_name"]
            country_code = tweet["place"]["country_code"]
            country = tweet["place"]["country"]
        else:
            place_type = None
            place_name = None
            place_full_name = None
            country_code = None
            country = None
            
        # Parsing profile geo data

        # First intialize profile geo field values as Nonetype for the default value (value if field not found in profile geo data)
        prof_country = None
        prof_country_code = None
        prof_locality = None
        prof_region = None
        prof_sub_region = None
        prof_full_name = None
        prof_coord_lat = None
        prof_coord_lng = None 

        # Parse desired profile geo data if the field is present 
        for field in tweet["user"]: 
            if field == "derived":
                prof_geo_data = tweet["user"]["derived"]
                if prof_geo_data["locations"] is not None:
                    for field in prof_geo_data["locations"][0]:
                        if field == "country":
                            prof_country = prof_geo_data["locations"][0][field]

                        elif field == "country_code":
                            prof_country_code = prof_geo_data["locations"][0][field]

                        elif field == "locality":
                            prof_locality = prof_geo_data["locations"][0][field]

                        elif field == "region":
                            prof_region = prof_geo_data["locations"][0][field]

                        elif field == "sub_region":
                            prof_sub_region = prof_geo_data["locations"][0][field]

                        elif field == "full_name":
                            prof_full_name = prof_geo_data["locations"][0][field]

                        elif field == "geo":
                            prof_coord_lat = prof_geo_data["locations"][0][field]["coordinates"][0]
                            prof_coord_lng = prof_geo_data["locations"][0][field]["coordinates"][1]

                        else:
                            pass
                else:
                    pass
            else:
                pass
            

            
        # Create dict with key-value pairs of parsed field and corresponding data
        mydict = { "tweet_id": tweet["id_str"], #Tweet ID
                       "date":tweet["created_at"], #Timestamp of tweet creation
                       "full_text": full_text, #Full tweet text
                       "tweet_type": determine_tweet_type(tweet), #Type of tweet
                       "hashtags": hashtags, #List of hashtags used in the tweet
                       "mentions": mentions, #List of mentions used in the tweet
                       "user_id": tweet["user"]["id_str"], #Twitter user profile ID
                       "user_location": tweet["user"]["location"], #The user's listed location
                       "geo": tweet["geo"], #Geodata 'geo' attribute
                       "lat_coordinates": coord_lat, #Geodata 'latitude coordinates' from 'coordinates' attribute
                       "lng_coordinates": coord_lng, #Geodata 'longitude coordinates' from 'coordinates' attribute
                       "place_type": place_type, #Geodata 'place_type' from 'place' attribute
                       "place_name": place_name, #Geodata 'name' from 'place' attribute
                       "place_full_name": place_full_name, #Geodata 'full_name' from 'place' attribute
                       "country_code": country_code, #Geodata 'country_code' from 'place' attribute
                       "country": country, #Geodata 'country' from 'place' attribute
                       "prof_country": prof_country, #Profile geodata 'country' from 'user''derived''locations'
                       "prof_country_code": prof_country_code, #Profile geodata 'country_code' from 'user''derived''locations'
                       "prof_locality": prof_locality, #Profile geodata 'locality' from 'user''derived''locations'
                       "prof_region": prof_region, #Profile geodata 'region' from 'user''derived''locations'
                       "prof_sub_region": prof_sub_region, #Profile geodata 'sub_region' from 'user''derived''locations'
                       "prof_full_name": prof_full_name, #Profile geodata 'full_name' from 'user''derived''locations'
                       "prof_coord_lat": prof_coord_lat, #Profile geodata 'latitude' from 'user''derived''locations''geo''coordinates'
                       "prof_coord_lng": prof_coord_lng, #Profile geodata 'longitude' from 'user''derived''locations''geo''coordinates'
                       "reply_count": tweet["reply_count"], #Number of times Tweet has been replied to
                       "quote_count": tweet["quote_count"], # Number of times Tweet has been quoted
                       "likes_count": tweet["favorite_count"], #Number of times Tweet has been liked 
                       "retweet_counts": tweet["retweet_count"], #Number of times this Tweet has been retweeted
                       "hyperlink": "https://twitter.com/twitter/status/" + tweet["id_str"] #Link to tweet
              }
        
        # Append parsed tweet data to list
        parsedTweets.append(mydict) # Add Tweet to parsedTweets list


In [3]:
# Read in raw JSON data file of 70K+ Tweets Pulled From the Twitter API
with open('../not_tracked/big_data_tweets_raw.json') as json_file:
    status = json.load(json_file)

In [4]:
# Parse the JSON data into a list of dictionaries called parsedTweets
parse_tweets(status)

In [5]:
# Create Pandas dataframe from parsedTweets list of dictionaries
tweet_df = pd.DataFrame(parsedTweets)

In [6]:
# Check tweets dataframe data types and missing values for each column
tweet_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72992 entries, 0 to 72991
Data columns (total 29 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   tweet_id           72992 non-null  object 
 1   date               72992 non-null  object 
 2   full_text          72992 non-null  object 
 3   tweet_type         72992 non-null  object 
 4   hashtags           10613 non-null  object 
 5   mentions           55129 non-null  object 
 6   user_id            72992 non-null  object 
 7   user_location      72794 non-null  object 
 8   geo                29 non-null     object 
 9   lat_coordinates    29 non-null     float64
 10  lng_coordinates    29 non-null     float64
 11  place_type         1973 non-null   object 
 12  place_name         1973 non-null   object 
 13  place_full_name    1973 non-null   object 
 14  country_code       1973 non-null   object 
 15  country            1973 non-null   object 
 16  prof_country       723

In [7]:
# drop 'geo', 'lat_coordinates', and 'lng_coordinates' columns for insufficient data
tweet_df.drop(['geo', 'lat_coordinates', 'lng_coordinates'], axis=1, inplace=True)
tweet_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72992 entries, 0 to 72991
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   tweet_id           72992 non-null  object 
 1   date               72992 non-null  object 
 2   full_text          72992 non-null  object 
 3   tweet_type         72992 non-null  object 
 4   hashtags           10613 non-null  object 
 5   mentions           55129 non-null  object 
 6   user_id            72992 non-null  object 
 7   user_location      72794 non-null  object 
 8   place_type         1973 non-null   object 
 9   place_name         1973 non-null   object 
 10  place_full_name    1973 non-null   object 
 11  country_code       1973 non-null   object 
 12  country            1973 non-null   object 
 13  prof_country       72392 non-null  object 
 14  prof_country_code  72392 non-null  object 
 15  prof_locality      39299 non-null  object 
 16  prof_region        630

In [8]:
# Check top 10 rows of tweets dataframe
tweet_df.head(10)

,tweet_id,date,full_text,tweet_type,hashtags,mentions,user_id,user_location,place_type,place_name,...,prof_region,prof_sub_region,prof_full_name,prof_coord_lat,prof_coord_lng,reply_count,quote_count,likes_count,retweet_counts,hyperlink
0,1588320083335467009,Thu Nov 03 23:59:52 +0000 2022,@UnnecRoughness A is Levis. \nB is KJ Jefferso...,Reply Tweet,None,[@UnnecRoughness],531999121,"Arkansas, USA",None,None,...,Arkansas,None,"Arkansas, United States",-92.50044,34.75037,0,0,6,0,https://twitter.com/twitter/status/15883200833...
1,1588320019259469824,Thu Nov 03 23:59:36 +0000 2022,Republican candidate's kids are almost KILLED ...,Original Tweet,None,None,1586027897579802624,United States,None,None,...,None,None,United States,-98.50000,39.76000,0,0,0,0,https://twitter.com/twitter/status/15883200192...
2,1588320006840160256,Thu Nov 03 23:59:33 +0000 2022,@Jupiter62214807 @cjstheman_611 @BMC_MacDaddy ...,Reply Tweet,None,"[@Jupiter62214807, @cjstheman_611, @BMC_MacDad...",783106891274596352,"Virginia, USA",None,None,...,West Virginia,None,"West Virginia, United States",-80.50009,38.50038,0,0,1,0,https://twitter.com/twitter/status/15883200068...
3,1588319891446272001,Thu Nov 03 23:59:06 +0000 2022,"@davidhogg111 It’s 2022, get new talking point...",Reply Tweet,None,[@davidhogg111],1511111650199412739,MI,None,None,...,Michigan,None,"Michigan, United States",-85.50033,44.25029,0,0,8,0,https://twitter.com/twitter/status/15883198914...
4,1588319866385477632,Thu Nov 03 23:59:00 +0000 2022,Watch: Gunman opens fire on Imran Khan https:/...,Original Tweet,None,None,287297482,USA,None,None,...,None,None,United States,-98.50000,39.76000,0,0,0,0,https://twitter.com/twitter/status/15883198663...
5,1588319827651104769,Thu Nov 03 23:58:51 +0000 2022,@Stormof69 @VoteYesOn114 I’m always amused tha...,Reply Tweet,None,"[@Stormof69, @VoteYesOn114]",412772316,"Manchester, CT",None,None,...,Connecticut,Hartford County,"Manchester, Connecticut, United States",-72.52148,41.77593,0,0,4,0,https://twitter.com/twitter/status/15883198276...
6,1588319810181529602,Thu Nov 03 23:58:46 +0000 2022,@ColinCowherd @DrHoodieMaddog What about kids ...,Reply Tweet,None,"[@ColinCowherd, @DrHoodieMaddog]",147647057,"Brooklyn, NY",None,None,...,New York,Kings County,"Brooklyn, New York, United States",-73.94958,40.65010,0,0,0,0,https://twitter.com/twitter/status/15883198101...
7,1588319807375433728,Thu Nov 03 23:58:46 +0000 2022,@Unnoble_Savage @brianleeellis @michaeljknowle...,Reply Tweet,None,"[@Unnoble_Savage, @brianleeellis, @michaeljkno...",1040679839747825664,"Willits, CA",None,None,...,California,Mendocino County,"Willits, California, United States",-123.35557,39.40961,0,0,0,0,https://twitter.com/twitter/status/15883198073...
8,1588319706343124992,Thu Nov 03 23:58:22 +0000 2022,@RobSchneider Let’s keep the 2nd Amendment and...,Reply Tweet,None,[@RobSchneider],1586158533149499392,"Monterey, CA",None,None,...,California,Monterey County,"Monterey, California, United States",-121.89468,36.60024,0,0,0,0,https://twitter.com/twitter/status/15883197063...
9,1588319697808019456,Thu Nov 03 23:58:20 +0000 2022,@davidhogg111 How many times a year are guns u...,Reply Tweet,None,[@davidhogg111],1527469162838253568,USA,None,None,...,None,None,United States,-98.50000,39.76000,0,0,0,0,https://twitter.com/twitter/status/15883196978...


In [9]:
# Confirm results for user.location and profile geo data parsing
tweet_df.loc[:, ['user_location', 'prof_country', 'prof_country_code', 'prof_locality', 'prof_region', 'prof_sub_region', 'prof_full_name']].head(10)

,user_location,prof_country,prof_country_code,prof_locality,prof_region,prof_sub_region,prof_full_name
0,"Arkansas, USA",United States,US,None,Arkansas,None,"Arkansas, United States"
1,United States,United States,US,None,None,None,United States
2,"Virginia, USA",United States,US,None,West Virginia,None,"West Virginia, United States"
3,MI,United States,US,None,Michigan,None,"Michigan, United States"
4,USA,United States,US,None,None,None,United States
5,"Manchester, CT",United States,US,Manchester,Connecticut,Hartford County,"Manchester, Connecticut, United States"
6,"Brooklyn, NY",United States,US,Brooklyn,New York,Kings County,"Brooklyn, New York, United States"
7,"Willits, CA",United States,US,Willits,California,Mendocino County,"Willits, California, United States"
8,"Monterey, CA",United States,US,Monterey,California,Monterey County,"Monterey, California, United States"
9,USA,United States,US,None,None,None,United States


In [10]:
# Check top 30 counties value counts
tweet_df['prof_sub_region'].value_counts()[0:30]

Los Angeles County      1774
Cook County             1534
Harris County           1100
Orange County           1040
Dallas County           1032
Multnomah County         911
Philadelphia County      866
Travis County            866
King County              818
Fulton County            734
Suffolk County           594
Maricopa County          579
Tarrant County           563
Bexar County             535
Clark County             491
San Diego County         450
Kings County             426
Jackson County           426
Miami-Dade County        425
Marion County            418
Palm Beach County        364
Allegheny County         354
Montgomery County        345
Denver County            330
San Francisco County     327
Jefferson County         326
Franklin County          321
Oklahoma County          311
City of Saint Louis      299
Hillsborough County      295
Name: prof_sub_region, dtype: int64

In [11]:
# Check top 30 cities values counts
tweet_df['prof_locality'].value_counts()[0:30]

Chicago              1405
Los Angeles          1281
Houston              1020
Dallas                949
Portland              936
New York City         935
Philadelphia          866
Austin                855
Atlanta               694
Seattle               642
Washington, D. C.     585
Florida               583
Boston                528
San Antonio           525
Fort Worth            474
Brooklyn              417
Phoenix               377
Las Vegas             367
San Diego             353
Miami                 343
Denver                331
San Francisco         327
Kansas City           325
Pittsburgh            321
St. Louis             299
Oklahoma City         298
Indianapolis          276
Tampa                 253
Charlotte             242
Columbus              242
Name: prof_locality, dtype: int64

In [12]:
# Check distribution of tweet types
tweet_df['tweet_type'].value_counts()

Reply Tweet       52264
Original Tweet    20720
Quote Tweet           8
Name: tweet_type, dtype: int64

In [13]:
# Remove Quote Tweet tweet types
tweet_df = tweet_df[tweet_df['tweet_type'] != "Quote Tweet"]

In [14]:
# Confirm removing Quote Tweets
tweet_df['tweet_type'].value_counts()

Reply Tweet       52264
Original Tweet    20720
Name: tweet_type, dtype: int64

In [15]:
# Check for unique values and respective counts in 'country_code' column
tweet_df['prof_country_code'].value_counts()

US    72331
GH       18
GB        6
ES        4
ZA        4
CD        2
VC        2
PH        2
TH        2
CN        2
KM        2
FR        1
BA        1
CF        1
AR        1
TN        1
GE        1
UY        1
MA        1
PG        1
Name: prof_country_code, dtype: int64

In [16]:
# Check for unique values and respective counts in 'prof_country' column
tweet_df['prof_country'].value_counts()

United States                             72331
Ghana                                        18
United Kingdom                                6
Spain                                         4
South Africa                                  4
Congo (The Democratic Republic of the)        2
Saint Vincent and the Grenadines              2
Philippines                                   2
Thailand                                      2
China                                         2
Comoros                                       2
France                                        1
Bosnia and Herzegovina                        1
Central African Republic                      1
Argentina                                     1
Tunisia                                       1
Georgia                                       1
Uruguay                                       1
Morocco                                       1
Papua New Guinea                              1
Name: prof_country, dtype: int64

In [17]:
# Check rows with non-null values in 'country_code' that are not 'US' for conflicts with profile geo data
tweet_df.loc[lambda df: (df['prof_country_code'] == 'GH') | (df['prof_country_code'] == 'GB') | (df['prof_country_code'] == 'ES')].T

,1295,14687,21775,42218,43442,47787,47940,57427,62924,68963,...,70222,70227,70238,70241,70264,70297,70307,70317,70354,71082
tweet_id,1588274567612694538,1589038041108451329,1589333929034940418,1590424008176340992,1590389813248921600,1590193957543505921,1590190511163592704,1590514576894099456,1590861393406078976,1591867895440887809,...,1591818222789353474,1591818104639733762,1591817689294835712,1591817591500464129,1591816591997820928,1591815094484480000,1591814735137280000,1591814456040099841,1591812529600315392,1591770598711099392
date,Thu Nov 03 20:59:00 +0000 2022,Sat Nov 05 23:32:46 +0000 2022,Sun Nov 06 19:08:31 +0000 2022,Wed Nov 09 19:20:06 +0000 2022,Wed Nov 09 17:04:14 +0000 2022,Wed Nov 09 04:05:58 +0000 2022,Wed Nov 09 03:52:16 +0000 2022,Thu Nov 10 01:20:00 +0000 2022,Fri Nov 11 00:18:07 +0000 2022,Sun Nov 13 18:57:36 +0000 2022,...,Sun Nov 13 15:40:13 +0000 2022,Sun Nov 13 15:39:45 +0000 2022,Sun Nov 13 15:38:06 +0000 2022,Sun Nov 13 15:37:43 +0000 2022,Sun Nov 13 15:33:44 +0000 2022,Sun Nov 13 15:27:47 +0000 2022,Sun Nov 13 15:26:22 +0000 2022,Sun Nov 13 15:25:15 +0000 2022,Sun Nov 13 15:17:36 +0000 2022,Sun Nov 13 12:30:59 +0000 2022
full_text,"'My teacher is dead': Girl, 10, trapped in sch...",@flashwizardjr @JoeBiden You do know our count...,@DeltaLowda @FordFischer @IamHawkNewsome @FBI ...,@captain_kityy @rayeahri @superwong18 @beatbya...,@MiKeysundrthmat @munchindaglizzy @Jellyjon_91...,@1andonlybrittne I think with the demographics...,@1andonlybrittne Running Beto for a third time...,@ElieNYC @davidhogg111 Funny how you care abou...,@davidhogg111 STOP #GunControl BS https://t.co...,@The1Hauntings @JimboUSA1776 @KeelKeel13 @Laca...,...,@Derrick47886845 @DianaHe50951535 @kali8989 @r...,@Derrick47886845 @DianaHe50951535 @kali8989 @r...,@Derrick47886845 @DianaHe50951535 @kali8989 @r...,@Derrick47886845 @DianaHe50951535 @kali8989 @r...,@Derrick47886845 @DianaHe50951535 @kali8989 @r...,@Derrick47886845 @DianaHe50951535 @kali8989 @r...,@Derrick47886845 @DianaHe50951535 @kali8989 @r...,@Derrick47886845 @DianaHe50951535 @kali8989 @r...,@Derrick47886845 @DianaHe50951535 @kali8989 @r...,@christycarruth @GavinClimie Gun ownership is ...
tweet_type,Original Tweet,Reply Tweet,Reply Tweet,Reply Tweet,Reply Tweet,Reply Tweet,Reply Tweet,Reply Tweet,Reply Tweet,Reply Tweet,...,Reply Tweet,Reply Tweet,Reply Tweet,Reply Tweet,Reply Tweet,Reply Tweet,Reply Tweet,Reply Tweet,Reply Tweet,Reply Tweet
hashtags,"[#Uvalde, #UvaldeMassacre, #uvaldestrong]",None,None,None,None,None,None,None,[#GunControl],None,...,None,None,None,None,None,None,None,"[#BackTheBlue, #BlueLine]",None,None
mentions,None,"[@flashwizardjr, @JoeBiden]","[@DeltaLowda, @FordFischer, @IamHawkNewsome, @...","[@captain_kityy, @rayeahri, @superwong18, @bea...","[@MiKeysundrthmat, @munchindaglizzy, @Jellyjon...",[@1andonlybrittne],[@1andonlybrittne],"[@ElieNYC, @davidhogg111]",[@davidhogg111],"[@The1Hauntings, @JimboUSA1776, @KeelKeel13, @...",...,"[@Derrick47886845, @DianaHe50951535, @kali8989...","[@Derrick47886845, @DianaHe50951535, @kali8989...","[@Derrick47886845, @DianaHe50951535, @kali8989...","[@Derrick47886845, @DianaHe50951535, @kali8989...","[@Derrick47886845, @DianaHe50951535, @kali8989...","[@Derrick47886845, @DianaHe50951535, @kali8989...","[@Derrick47886845, @DianaHe50951535, @kali8989...","[@Derrick47886845, @DianaHe50951535, @kali8989...","[@Derrick47886845, @DianaHe50951535, @kali8989...","[@christycarruth, @GavinClimie]"
user_id,17895820,1555116935381237762,22033036,1555116935381237762,1555116935381237762,1519163177711321088,1519163177711321088,1555116935381237762,26282272,105926399,...,105926399,105926399,105926399,105926399,105926399,105926399,105926399,105926399,105926399,1545103667124224001
user_location,London,"Minas Tirith, Gondor",United Kingdom,"Minas Tirith, Gondor","Minas Tirith, Gondor",Vatican City,Vatican City,"Minas Tirith, Gondor",Saturn,"Accra, Ghana",...,"Accra, Ghana","Accra, Ghana","Accra, Ghana","Accra, Ghana","Accra, Ghana","A

In [18]:
# Filter for only rows where prof_country_code is 'US'
tweet_df = tweet_df[tweet_df['prof_country_code'] == 'US']
# return value counts of 'prof_country_code' and 'prof_country' to confirm only tweets from US profiles exist in data
print(f"Country Code Counts:\n {tweet_df['prof_country_code'].value_counts()}\n\nCountry Name Counts:\n {tweet_df['prof_country'].value_counts()}")

Country Code Counts:
 US    72331
Name: prof_country_code, dtype: int64

Country Name Counts:
 United States    72331
Name: prof_country, dtype: int64


In [19]:
# Drop 'prof_country_code'
tweet_df.drop(['prof_country_code'], axis=1, inplace=True)

In [20]:
# Check top 30 user locations value counts
tweet_df['user_location'].value_counts()[0:30]

United States        4764
USA                  2581
Texas, USA           1605
Florida, USA         1205
California, USA      1097
Washington, DC       1097
Texas                1010
Chicago, IL           976
Los Angeles, CA       763
Houston, TX           735
New York, NY          699
New York, USA         693
Dallas, TX            655
Portland, OR          611
Pennsylvania, USA     590
Philadelphia, PA      588
Austin, TX            557
New Jersey, USA       504
Washington, USA       473
Atlanta, GA           472
New York City         462
Fort Worth, TX        426
New York              424
Seattle, WA           420
Michigan, USA         405
Ohio, USA             395
San Antonio, TX       384
California            355
Arizona, USA          352
Colorado, USA         350
Name: user_location, dtype: int64

In [21]:
# Check top 30 prof_full_name value counts
tweet_df['prof_full_name'].value_counts()[0:30]

United States                                         9365
Texas, United States                                  3268
New York, United States                               2955
California, United States                             1732
Washington, D.C., United States                       1563
Chicago, Illinois, United States                      1405
Los Angeles, California, United States                1280
Florida, United States                                1231
New Jersey, United States                             1030
Houston, Texas, United States                         1018
Dallas, Texas, United States                           947
New York City, New York, United States                 935
Pennsylvania, United States                            918
Portland, Oregon, United States                        906
Philadelphia, Pennsylvania, United States              866
Austin, Texas, United States                           854
Ohio, United States                                    7

In [22]:
tweet_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 72331 entries, 0 to 72991
Data columns (total 25 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   tweet_id         72331 non-null  object 
 1   date             72331 non-null  object 
 2   full_text        72331 non-null  object 
 3   tweet_type       72331 non-null  object 
 4   hashtags         10552 non-null  object 
 5   mentions         54574 non-null  object 
 6   user_id          72331 non-null  object 
 7   user_location    72331 non-null  object 
 8   place_type       1964 non-null   object 
 9   place_name       1964 non-null   object 
 10  place_full_name  1964 non-null   object 
 11  country_code     1964 non-null   object 
 12  country          1964 non-null   object 
 13  prof_country     72331 non-null  object 
 14  prof_locality    39278 non-null  object 
 15  prof_region      62966 non-null  object 
 16  prof_sub_region  37758 non-null  object 
 17  prof_full_na

In [23]:
# drop all columns that are redundant and/ or contain more null values than other column carrying the same categorical information
tweet_df.drop(['user_location', 'place_type', 'place_name', 'place_full_name', 'country_code', 'country', 'prof_full_name'], axis=1, inplace=True)

In [24]:
tweet_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 72331 entries, 0 to 72991
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   tweet_id         72331 non-null  object 
 1   date             72331 non-null  object 
 2   full_text        72331 non-null  object 
 3   tweet_type       72331 non-null  object 
 4   hashtags         10552 non-null  object 
 5   mentions         54574 non-null  object 
 6   user_id          72331 non-null  object 
 7   prof_country     72331 non-null  object 
 8   prof_locality    39278 non-null  object 
 9   prof_region      62966 non-null  object 
 10  prof_sub_region  37758 non-null  object 
 11  prof_coord_lat   72331 non-null  float64
 12  prof_coord_lng   72331 non-null  float64
 13  reply_count      72331 non-null  int64  
 14  quote_count      72331 non-null  int64  
 15  likes_count      72331 non-null  int64  
 16  retweet_counts   72331 non-null  int64  
 17  hyperlink   

In [25]:
# Rename columns for ease of interpretation
tweet_df.rename(columns={'prof_country': 'country', 'prof_locality': 'city', 'prof_region': 'state', 'prof_sub_region': 'county', 'prof_coord_lat': 'latitude', 'prof_coord_lng': 'longitude'}, inplace=True)

In [26]:
# Confirm column name changes
tweet_df.columns

Index(['tweet_id', 'date', 'full_text', 'tweet_type', 'hashtags', 'mentions',
       'user_id', 'country', 'city', 'state', 'county', 'latitude',
       'longitude', 'reply_count', 'quote_count', 'likes_count',
       'retweet_counts', 'hyperlink'],
      dtype='object')

In [27]:
# Change order of columns
tweet_df = tweet_df.loc[:,['tweet_id', 'date', 'full_text', 'tweet_type', 'hashtags', 'mentions',
                'user_id', 'city', 'county', 'state', 'country', 'latitude',
                'longitude', 'reply_count', 'quote_count', 'likes_count',
                'retweet_counts', 'hyperlink']]

In [28]:
# Confirm column changes with corrects row values
tweet_df.head(10)

,tweet_id,date,full_text,tweet_type,hashtags,mentions,user_id,city,county,state,country,latitude,longitude,reply_count,quote_count,likes_count,retweet_counts,hyperlink
0,1588320083335467009,Thu Nov 03 23:59:52 +0000 2022,@UnnecRoughness A is Levis. \nB is KJ Jefferso...,Reply Tweet,None,[@UnnecRoughness],531999121,None,None,Arkansas,United States,-92.50044,34.75037,0,0,6,0,https://twitter.com/twitter/status/15883200833...
1,1588320019259469824,Thu Nov 03 23:59:36 +0000 2022,Republican candidate's kids are almost KILLED ...,Original Tweet,None,None,1586027897579802624,None,None,None,United States,-98.50000,39.76000,0,0,0,0,https://twitter.com/twitter/status/15883200192...
2,1588320006840160256,Thu Nov 03 23:59:33 +0000 2022,@Jupiter62214807 @cjstheman_611 @BMC_MacDaddy ...,Reply Tweet,None,"[@Jupiter62214807, @cjstheman_611, @BMC_MacDad...",783106891274596352,None,None,West Virginia,United States,-80.50009,38.50038,0,0,1,0,https://twitter.com/twitter/status/15883200068...
3,1588319891446272001,Thu Nov 03 23:59:06 +0000 2022,"@davidhogg111 It’s 2022, get new talking point...",Reply Tweet,None,[@davidhogg111],1511111650199412739,None,None,Michigan,United States,-85.50033,44.25029,0,0,8,0,https://twitter.com/twitter/status/15883198914...
4,1588319866385477632,Thu Nov 03 23:59:00 +0000 2022,Watch: Gunman opens fire on Imran Khan https:/...,Original Tweet,None,None,287297482,None,None,None,United States,-98.50000,39.76000,0,0,0,0,https://twitter.com/twitter/status/15883198663...
5,1588319827651104769,Thu Nov 03 23:58:51 +0000 2022,@Stormof69 @VoteYesOn114 I’m always amused tha...,Reply Tweet,None,"[@Stormof69, @VoteYesOn114]",412772316,Manchester,Hartford County,Connecticut,United States,-72.52148,41.77593,0,0,4,0,https://twitter.com/twitter/status/15883198276...
6,1588319810181529602,Thu Nov 03 23:58:46 +0000 2022,@ColinCowherd @DrHoodieMaddog What about kids ...,Reply Tweet,None,"[@ColinCowherd, @DrHoodieMaddog]",147647057,Brooklyn,Kings County,New York,United States,-73.94958,40.65010,0,0,0,0,https://twitter.com/twitter/status/15883198101...
7,1588319807375433728,Thu Nov 03 23:58:46 +0000 2022,@Unnoble_Savage @brianleeellis @michaeljknowle...,Reply Tweet,None,"[@Unnoble_Savage, @brianleeellis, @michaeljkno...",1040679839747825664,Willits,Mendocino County,California,United States,-123.35557,39.40961,0,0,0,0,https://twitter.com/twitter/status/15883198073...
8,1588319706343124992,Thu Nov 03 23:58:22 +0000 2022,@RobSchneider Let’s keep the 2nd Amendment and...,Reply Tweet,None,[@RobSchneider],1586158533149499392,Monterey,Monterey County,California,United States,-121.89468,36.60024,0,0,0,0,https://twitter.com/twitter/status/15883197063...
9,1588319697808019456,Thu Nov 03 23:58:20 +0000 2022,@davidhogg111 How many times a year are guns u...,Reply Tweet,None,[@davidhogg111],1527469162838253568,None,None,None,United States,-98.50000,39.76000,0,0,0,0,https://twitter.com/twitter/status/15883196978...


In [29]:
tweet_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 72331 entries, 0 to 72991
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tweet_id        72331 non-null  object 
 1   date            72331 non-null  object 
 2   full_text       72331 non-null  object 
 3   tweet_type      72331 non-null  object 
 4   hashtags        10552 non-null  object 
 5   mentions        54574 non-null  object 
 6   user_id         72331 non-null  object 
 7   city            39278 non-null  object 
 8   county          37758 non-null  object 
 9   state           62966 non-null  object 
 10  country         72331 non-null  object 
 11  latitude        72331 non-null  float64
 12  longitude       72331 non-null  float64
 13  reply_count     72331 non-null  int64  
 14  quote_count     72331 non-null  int64  
 15  likes_count     72331 non-null  int64  
 16  retweet_counts  72331 non-null  int64  
 17  hyperlink       72331 non-null 

In [30]:
# Create 'date_created' column with datetime conversion of 'date' column values
tweet_df['date_created'] = tweet_df['date'].apply(lambda x: datetime.strftime(datetime.strptime(x, '%a %b %d %H:%M:%S +0000 %Y'), '%Y-%m-%d %H:%M:%S'))

tweet_df.loc[:, ['date', 'date_created']]

,date,date_created
0,Thu Nov 03 23:59:52 +0000 2022,2022-11-03 23:59:52
1,Thu Nov 03 23:59:36 +0000 2022,2022-11-03 23:59:36
2,Thu Nov 03 23:59:33 +0000 2022,2022-11-03 23:59:33
3,Thu Nov 03 23:59:06 +0000 2022,2022-11-03 23:59:06
4,Thu Nov 03 23:59:00 +0000 2022,2022-11-03 23:59:00
...,...,...
72987,Sun Nov 13 00:00:13 +0000 2022,2022-11-13 00:00:13
72988,Sun Nov 13 00:00:11 +0000 2022,2022-11-13 00:00:11
72989,Sun Nov 13 00:00:03 +0000 2022,2022-11-13 00:00:03
72990,Sun Nov 13 00:00:02 +0000 2022,2022-11-13 00:00:02


In [31]:
tweet_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 72331 entries, 0 to 72991
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tweet_id        72331 non-null  object 
 1   date            72331 non-null  object 
 2   full_text       72331 non-null  object 
 3   tweet_type      72331 non-null  object 
 4   hashtags        10552 non-null  object 
 5   mentions        54574 non-null  object 
 6   user_id         72331 non-null  object 
 7   city            39278 non-null  object 
 8   county          37758 non-null  object 
 9   state           62966 non-null  object 
 10  country         72331 non-null  object 
 11  latitude        72331 non-null  float64
 12  longitude       72331 non-null  float64
 13  reply_count     72331 non-null  int64  
 14  quote_count     72331 non-null  int64  
 15  likes_count     72331 non-null  int64  
 16  retweet_counts  72331 non-null  int64  
 17  hyperlink       72331 non-null 

In [32]:
# Change order of columns by moving new 'date_created' column to 'date' column position and exclude 'date' column 
tweet_df = tweet_df.loc[:,['tweet_id', 'date_created', 'full_text', 'tweet_type', 'hashtags', 'mentions',
                'user_id', 'city', 'county', 'state', 'country', 'latitude',
                'longitude', 'reply_count', 'quote_count', 'likes_count',
                'retweet_counts', 'hyperlink']]

In [33]:
# Confirm changes to columns
tweet_df.head(10)

,tweet_id,date_created,full_text,tweet_type,hashtags,mentions,user_id,city,county,state,country,latitude,longitude,reply_count,quote_count,likes_count,retweet_counts,hyperlink
0,1588320083335467009,2022-11-03 23:59:52,@UnnecRoughness A is Levis. \nB is KJ Jefferso...,Reply Tweet,None,[@UnnecRoughness],531999121,None,None,Arkansas,United States,-92.50044,34.75037,0,0,6,0,https://twitter.com/twitter/status/15883200833...
1,1588320019259469824,2022-11-03 23:59:36,Republican candidate's kids are almost KILLED ...,Original Tweet,None,None,1586027897579802624,None,None,None,United States,-98.50000,39.76000,0,0,0,0,https://twitter.com/twitter/status/15883200192...
2,1588320006840160256,2022-11-03 23:59:33,@Jupiter62214807 @cjstheman_611 @BMC_MacDaddy ...,Reply Tweet,None,"[@Jupiter62214807, @cjstheman_611, @BMC_MacDad...",783106891274596352,None,None,West Virginia,United States,-80.50009,38.50038,0,0,1,0,https://twitter.com/twitter/status/15883200068...
3,1588319891446272001,2022-11-03 23:59:06,"@davidhogg111 It’s 2022, get new talking point...",Reply Tweet,None,[@davidhogg111],1511111650199412739,None,None,Michigan,United States,-85.50033,44.25029,0,0,8,0,https://twitter.com/twitter/status/15883198914...
4,1588319866385477632,2022-11-03 23:59:00,Watch: Gunman opens fire on Imran Khan https:/...,Original Tweet,None,None,287297482,None,None,None,United States,-98.50000,39.76000,0,0,0,0,https://twitter.com/twitter/status/15883198663...
5,1588319827651104769,2022-11-03 23:58:51,@Stormof69 @VoteYesOn114 I’m always amused tha...,Reply Tweet,None,"[@Stormof69, @VoteYesOn114]",412772316,Manchester,Hartford County,Connecticut,United States,-72.52148,41.77593,0,0,4,0,https://twitter.com/twitter/status/15883198276...
6,1588319810181529602,2022-11-03 23:58:46,@ColinCowherd @DrHoodieMaddog What about kids ...,Reply Tweet,None,"[@ColinCowherd, @DrHoodieMaddog]",147647057,Brooklyn,Kings County,New York,United States,-73.94958,40.65010,0,0,0,0,https://twitter.com/twitter/status/15883198101...
7,1588319807375433728,2022-11-03 23:58:46,@Unnoble_Savage @brianleeellis @michaeljknowle...,Reply Tweet,None,"[@Unnoble_Savage, @brianleeellis, @michaeljkno...",1040679839747825664,Willits,Mendocino County,California,United States,-123.35557,39.40961,0,0,0,0,https://twitter.com/twitter/status/15883198073...
8,1588319706343124992,2022-11-03 23:58:22,@RobSchneider Let’s keep the 2nd Amendment and...,Reply Tweet,None,[@RobSchneider],1586158533149499392,Monterey,Monterey County,California,United States,-121.89468,36.60024,0,0,0,0,https://twitter.com/twitter/status/15883197063...
9,1588319697808019456,2022-11-03 23:58:20,@davidhogg111 How many times a year are guns u...,Reply Tweet,None,[@davidhogg111],1527469162838253568,None,None,None,United States,-98.50000,39.76000,0,0,0,0,https://twitter.com/twitter/status/15883196978...


In [34]:
# Create dummy_sentiment column for mockup visualizations
tweet_df['dummy_sentiment'] = np.random.choice(['anti-gun', 'pro-gun', 'neutral'], size=len(tweet_df))

# Confirm dummy column creation
tweet_df.head(10)

,tweet_id,date_created,full_text,tweet_type,hashtags,mentions,user_id,city,county,state,country,latitude,longitude,reply_count,quote_count,likes_count,retweet_counts,hyperlink,dummy_sentiment
0,1588320083335467009,2022-11-03 23:59:52,@UnnecRoughness A is Levis. \nB is KJ Jefferso...,Reply Tweet,None,[@UnnecRoughness],531999121,None,None,Arkansas,United States,-92.50044,34.75037,0,0,6,0,https://twitter.com/twitter/status/15883200833...,pro-gun
1,1588320019259469824,2022-11-03 23:59:36,Republican candidate's kids are almost KILLED ...,Original Tweet,None,None,1586027897579802624,None,None,None,United States,-98.50000,39.76000,0,0,0,0,https://twitter.com/twitter/status/15883200192...,pro-gun
2,1588320006840160256,2022-11-03 23:59:33,@Jupiter62214807 @cjstheman_611 @BMC_MacDaddy ...,Reply Tweet,None,"[@Jupiter62214807, @cjstheman_611, @BMC_MacDad...",783106891274596352,None,None,West Virginia,United States,-80.50009,38.50038,0,0,1,0,https://twitter.com/twitter/status/15883200068...,anti-gun
3,1588319891446272001,2022-11-03 23:59:06,"@davidhogg111 It’s 2022, get new talking point...",Reply Tweet,None,[@davidhogg111],1511111650199412739,None,None,Michigan,United States,-85.50033,44.25029,0,0,8,0,https://twitter.com/twitter/status/15883198914...,neutral
4,1588319866385477632,2022-11-03 23:59:00,Watch: Gunman opens fire on Imran Khan https:/...,Original Tweet,None,None,287297482,None,None,None,United States,-98.50000,39.76000,0,0,0,0,https://twitter.com/twitter/status/15883198663...,pro-gun
5,1588319827651104769,2022-11-03 23:58:51,@Stormof69 @VoteYesOn114 I’m always amused tha...,Reply Tweet,None,"[@Stormof69, @VoteYesOn114]",412772316,Manchester,Hartford County,Connecticut,United States,-72.52148,41.77593,0,0,4,0,https://twitter.com/twitter/status/15883198276...,pro-gun
6,1588319810181529602,2022-11-03 23:58:46,@ColinCowherd @DrHoodieMaddog What about kids ...,Reply Tweet,None,"[@ColinCowherd, @DrHoodieMaddog]",147647057,Brooklyn,Kings County,New York,United States,-73.94958,40.65010,0,0,0,0,https://twitter.com/twitter/status/15883198101...,neutral
7,1588319807375433728,2022-11-03 23:58:46,@Unnoble_Savage @brianleeellis @michaeljknowle...,Reply Tweet,None,"[@Unnoble_Savage, @brianleeellis, @michaeljkno...",1040679839747825664,Willits,Mendocino County,California,United States,-123.35557,39.40961,0,0,0,0,https://twitter.com/twitter/status/15883198073...,anti-gun
8,1588319706343124992,2022-11-03 23:58:22,@RobSchneider Let’s keep the 2nd Amendment and...,Reply Tweet,None,[@RobSchneider],1586158533149499392,Monterey,Monterey County,California,United States,-121.89468,36.60024,0,0,0,0,https://twitter.com/twitter/status/15883197063...,anti-gun
9,1588319697808019456,2022-11-03 23:58:20,@davidhogg111 How many times a year are guns u...,Reply Tweet,None,[@davidhogg111],1527469162838253568,None,None,None,United States,-98.50000,39.76000,0,0,0,0,https://twitter.com/twitter/status/15883196978...,pro-gun


In [35]:
tweet_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 72331 entries, 0 to 72991
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   tweet_id         72331 non-null  object 
 1   date_created     72331 non-null  object 
 2   full_text        72331 non-null  object 
 3   tweet_type       72331 non-null  object 
 4   hashtags         10552 non-null  object 
 5   mentions         54574 non-null  object 
 6   user_id          72331 non-null  object 
 7   city             39278 non-null  object 
 8   county           37758 non-null  object 
 9   state            62966 non-null  object 
 10  country          72331 non-null  object 
 11  latitude         72331 non-null  float64
 12  longitude        72331 non-null  float64
 13  reply_count      72331 non-null  int64  
 14  quote_count      72331 non-null  int64  
 15  likes_count      72331 non-null  int64  
 16  retweet_counts   72331 non-null  int64  
 17  hyperlink   

In [36]:
# Check random distribuiton of dummy_sentiment values
tweet_df['dummy_sentiment'].value_counts()

anti-gun    24228
pro-gun     24120
neutral     23983
Name: dummy_sentiment, dtype: int64

In [37]:
# Set up AddFIPS API
af = addfips.AddFIPS()

# Add State FIPS codes to tweet_df
tweet_df['fips_state'] = tweet_df['state'].apply(lambda x: af.get_state_fips(x))

tweet_df.loc[:, ['state', 'fips_state']]

,state,fips_state
0,Arkansas,05
1,None,None
2,West Virginia,54
3,Michigan,26
4,None,None
...,...,...
72987,Missouri,29
72988,New York,36
72989,Ohio,39
72990,Georgia,13


In [38]:
# See which non-null states did not return a fips
tweet_df.loc[tweet_df['fips_state'].isna()][['state']].dropna().value_counts()

state           
Washington, D.C.    2148
dtype: int64

In [39]:
# Confirm alternate naming yields fips from API for "Washington, D.C."
af.get_state_fips('D.C.')

'11'

In [40]:
# Save D.C. fips to variable
DC = af.get_state_fips('D.C.')

# Replace null value fips for "Washington, D.C." with the corresponding state fips code ("11")
tweet_df.loc[tweet_df['state'] == 'Washington, D.C.', 'fips_state'] = DC

# Confirm all necessary changes were made
tweet_df.loc[tweet_df['state'] == 'Washington, D.C.'][['fips_state']].value_counts()

fips_state
11            2148
dtype: int64

In [41]:
tweet_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 72331 entries, 0 to 72991
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   tweet_id         72331 non-null  object 
 1   date_created     72331 non-null  object 
 2   full_text        72331 non-null  object 
 3   tweet_type       72331 non-null  object 
 4   hashtags         10552 non-null  object 
 5   mentions         54574 non-null  object 
 6   user_id          72331 non-null  object 
 7   city             39278 non-null  object 
 8   county           37758 non-null  object 
 9   state            62966 non-null  object 
 10  country          72331 non-null  object 
 11  latitude         72331 non-null  float64
 12  longitude        72331 non-null  float64
 13  reply_count      72331 non-null  int64  
 14  quote_count      72331 non-null  int64  
 15  likes_count      72331 non-null  int64  
 16  retweet_counts   72331 non-null  int64  
 17  hyperlink   

In [42]:
# Add County FIPS codes to tweet_df
tweet_df['fips_county'] = tweet_df.apply(lambda x: af.get_county_fips(x['county'], state=x['state']) if x['county'] is not None else None, axis=1)

tweet_df.loc[:, ['county', 'fips_county']]

,county,fips_county
0,None,None
1,None,None
2,None,None
3,None,None
4,None,None
...,...,...
72987,Webster County,29225
72988,None,None
72989,Hamilton County,39061
72990,Fulton County,13121


In [43]:
# Check if any non-null counties did not return a fips
tweet_df.loc[tweet_df['fips_county'].isna()][['county']].dropna().value_counts()

county                  
City of Saint Louis         299
City of Baltimore           214
City of Fairfax             140
City of Virginia Beach      115
City of Richmond             90
City of Roanoke              80
City of Charlottesville      33
City of Alexandria           30
City of Fredericksburg       19
City of Norfolk              18
City of Chesapeake           12
City of Lynchburg            11
City of Newport News         10
City of Williamsburg          7
City of Winchester            6
City of Hampton               6
City of Suffolk               6
City of Manassas              6
City of Harrisonburg          4
City of Salem                 3
City of Falls Church          3
City of Emporia               3
City of Portsmouth            2
City of Martinsville          2
City of Lexington             2
City of Danville              1
City of Covington             1
City of Colonial Heights      1
City of Bristol               1
City of Bedford               1
City of Waynesb

In [44]:
# Confirm there is only 299 'City of Saint Louis' for testing example
tweet_df.loc[tweet_df['county'] == 'City of Saint Louis']['county'].value_counts()

City of Saint Louis    299
Name: county, dtype: int64

In [45]:
# Replace "City of " with empy string to remove the naming convention from 'county' column
tweet_df['county'] = tweet_df['county'].apply(lambda x: x.replace("City of ", "") if x is not None else None)

# Confirm results against prior example
tweet_df.loc[tweet_df['county'] == 'Saint Louis']['county']

105      Saint Louis
777      Saint Louis
793      Saint Louis
963      Saint Louis
1242     Saint Louis
            ...     
71460    Saint Louis
71478    Saint Louis
71825    Saint Louis
72682    Saint Louis
72955    Saint Louis
Name: county, Length: 299, dtype: object

In [46]:
# Re-run code for getting county fips codes after cleaning county names with "City of "
tweet_df['fips_county'] = tweet_df.apply(lambda x: af.get_county_fips(x['county'], state=x['state']) if x['county'] is not None else None, axis=1)

tweet_df.loc[:, ['county', 'fips_county']]

,county,fips_county
0,None,None
1,None,None
2,None,None
3,None,None
4,None,None
...,...,...
72987,Webster County,29225
72988,None,None
72989,Hamilton County,39061
72990,Fulton County,13121


In [47]:
# Check if any non-null counties did not return a fips
tweet_df.loc[tweet_df['fips_county'].isna()][['county']].dropna().value_counts()

Series([], dtype: int64)

In [48]:
tweet_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 72331 entries, 0 to 72991
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   tweet_id         72331 non-null  object 
 1   date_created     72331 non-null  object 
 2   full_text        72331 non-null  object 
 3   tweet_type       72331 non-null  object 
 4   hashtags         10552 non-null  object 
 5   mentions         54574 non-null  object 
 6   user_id          72331 non-null  object 
 7   city             39278 non-null  object 
 8   county           37758 non-null  object 
 9   state            62966 non-null  object 
 10  country          72331 non-null  object 
 11  latitude         72331 non-null  float64
 12  longitude        72331 non-null  float64
 13  reply_count      72331 non-null  int64  
 14  quote_count      72331 non-null  int64  
 15  likes_count      72331 non-null  int64  
 16  retweet_counts   72331 non-null  int64  
 17  hyperlink   

In [49]:
# Change order of columns by moving 'fips_county' and 'fips_state' columns 
tweet_df = tweet_df.loc[:,['tweet_id', 'date_created', 'full_text', 'tweet_type', 'hashtags', 'mentions',
                'user_id', 'city', 'county', 'fips_county', 'state', 'fips_state', 'country', 'latitude',
                'longitude', 'reply_count', 'quote_count', 'likes_count',
                'retweet_counts', 'hyperlink', 'dummy_sentiment']]

In [50]:
# Final check of dataframe columns, non-null value counts and dtypes
tweet_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 72331 entries, 0 to 72991
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   tweet_id         72331 non-null  object 
 1   date_created     72331 non-null  object 
 2   full_text        72331 non-null  object 
 3   tweet_type       72331 non-null  object 
 4   hashtags         10552 non-null  object 
 5   mentions         54574 non-null  object 
 6   user_id          72331 non-null  object 
 7   city             39278 non-null  object 
 8   county           37758 non-null  object 
 9   fips_county      37758 non-null  object 
 10  state            62966 non-null  object 
 11  fips_state       62966 non-null  object 
 12  country          72331 non-null  object 
 13  latitude         72331 non-null  float64
 14  longitude        72331 non-null  float64
 15  reply_count      72331 non-null  int64  
 16  quote_count      72331 non-null  int64  
 17  likes_count 

In [51]:
# Check dataframe
tweet_df.head(10)

,tweet_id,date_created,full_text,tweet_type,hashtags,mentions,user_id,city,county,fips_county,...,fips_state,country,latitude,longitude,reply_count,quote_count,likes_count,retweet_counts,hyperlink,dummy_sentiment
0,1588320083335467009,2022-11-03 23:59:52,@UnnecRoughness A is Levis. \nB is KJ Jefferso...,Reply Tweet,None,[@UnnecRoughness],531999121,None,None,None,...,05,United States,-92.50044,34.75037,0,0,6,0,https://twitter.com/twitter/status/15883200833...,pro-gun
1,1588320019259469824,2022-11-03 23:59:36,Republican candidate's kids are almost KILLED ...,Original Tweet,None,None,1586027897579802624,None,None,None,...,None,United States,-98.50000,39.76000,0,0,0,0,https://twitter.com/twitter/status/15883200192...,pro-gun
2,1588320006840160256,2022-11-03 23:59:33,@Jupiter62214807 @cjstheman_611 @BMC_MacDaddy ...,Reply Tweet,None,"[@Jupiter62214807, @cjstheman_611, @BMC_MacDad...",783106891274596352,None,None,None,...,54,United States,-80.50009,38.50038,0,0,1,0,https://twitter.com/twitter/status/15883200068...,anti-gun
3,1588319891446272001,2022-11-03 23:59:06,"@davidhogg111 It’s 2022, get new talking point...",Reply Tweet,None,[@davidhogg111],1511111650199412739,None,None,None,...,26,United States,-85.50033,44.25029,0,0,8,0,https://twitter.com/twitter/status/15883198914...,neutral
4,1588319866385477632,2022-11-03 23:59:00,Watch: Gunman opens fire on Imran Khan https:/...,Original Tweet,None,None,287297482,None,None,None,...,None,United States,-98.50000,39.76000,0,0,0,0,https://twitter.com/twitter/status/15883198663...,pro-gun
5,1588319827651104769,2022-11-03 23:58:51,@Stormof69 @VoteYesOn114 I’m always amused tha...,Reply Tweet,None,"[@Stormof69, @VoteYesOn114]",412772316,Manchester,Hartford County,09003,...,09,United States,-72.52148,41.77593,0,0,4,0,https://twitter.com/twitter/status/15883198276...,pro-gun
6,1588319810181529602,2022-11-03 23:58:46,@ColinCowherd @DrHoodieMaddog What about kids ...,Reply Tweet,None,"[@ColinCowherd, @DrHoodieMaddog]",147647057,Brooklyn,Kings County,36047,...,36,United States,-73.94958,40.65010,0,0,0,0,https://twitter.com/twitter/status/15883198101...,neutral
7,1588319807375433728,2022-11-03 23:58:46,@Unnoble_Savage @brianleeellis @michaeljknowle...,Reply Tweet,None,"[@Unnoble_Savage, @brianleeellis, @michaeljkno...",1040679839747825664,Willits,Mendocino County,06045,...,06,United States,-123.35557,39.40961,0,0,0,0,https://twitter.com/twitter/status/15883198073...,anti-gun
8,1588319706343124992,2022-11-03 23:58:22,@RobSchneider Let’s keep the 2nd Amendment and...,Reply Tweet,None,[@RobSchneider],1586158533149499392,Monterey,Monterey County,06053,...,06,United States,-121.89468,36.60024,0,0,0,0,https://twitter.com/twitter/status/15883197063...,anti-gun
9,1588319697808019456,2022-11-03 23:58:20,@davidhogg111 How many times a year are guns u...,Reply Tweet,None,[@davidhogg111],1527469162838253568,None,None,None,...,None,United States,-98.50000,39.76000,0,0,0,0,https://twitter.com/twitter/status/15883196978...,pro-gun


In [52]:
# Wrap every tweet in "full_text" column to avoid delimiting errors upon writing to CSV file
tweet_df['full_text'] = '"' + tweet_df['full_text'] + '"'

In [59]:
# Confirm final dataframe
tweet_df.head()

,tweet_id,date_created,full_text,tweet_type,hashtags,mentions,user_id,city,county,fips_county,...,fips_state,country,latitude,longitude,reply_count,quote_count,likes_count,retweet_counts,hyperlink,dummy_sentiment
0,1588320083335467009,2022-11-03 23:59:52,"""@UnnecRoughness A is Levis. \nB is KJ Jeffers...",Reply Tweet,None,[@UnnecRoughness],531999121,None,None,None,...,05,United States,-92.50044,34.75037,0,0,6,0,https://twitter.com/twitter/status/15883200833...,pro-gun
1,1588320019259469824,2022-11-03 23:59:36,"""Republican candidate's kids are almost KILLED...",Original Tweet,None,None,1586027897579802624,None,None,None,...,NaN,United States,-98.50000,39.76000,0,0,0,0,https://twitter.com/twitter/status/15883200192...,pro-gun
2,1588320006840160256,2022-11-03 23:59:33,"""@Jupiter62214807 @cjstheman_611 @BMC_MacDaddy...",Reply Tweet,None,"[@Jupiter62214807, @cjstheman_611, @BMC_MacDad...",783106891274596352,None,None,None,...,54,United States,-80.50009,38.50038,0,0,1,0,https://twitter.com/twitter/status/15883200068...,anti-gun
3,1588319891446272001,2022-11-03 23:59:06,"""@davidhogg111 It’s 2022, get new talking poin...",Reply Tweet,None,[@davidhogg111],1511111650199412739,None,None,None,...,26,United States,-85.50033,44.25029,0,0,8,0,https://twitter.com/twitter/status/15883198914...,neutral
4,1588319866385477632,2022-11-03 23:59:00,"""Watch: Gunman opens fire on Imran Khan https:...",Original Tweet,None,None,287297482,None,None,None,...,NaN,United States,-98.50000,39.76000,0,0,0,0,https://twitter.com/twitter/status/15883198663...,pro-gun


In [61]:
# Export tweet_df to CSV
tweet_df.to_csv("../res/big_data_tweets.csv", index=False)